In [1]:
import pandas as pd
import numpy as  np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

In [3]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [5]:
path = '/content/the_tree.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 64808


In [7]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [10]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [16]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[36])
print(next_words[6])

['you', 'he', 'said', 'aiken', 'frowned']
but


In [17]:
prev_words[:5]

[['the', 'tree', 'by', 'k', 'm'],
 ['tree', 'by', 'k', 'm', 'fox'],
 ['by', 'k', 'm', 'fox', 'the'],
 ['k', 'm', 'fox', 'the', 'tree'],
 ['m', 'fox', 'the', 'tree', 'chapter']]

In [19]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)

In [20]:
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [26]:
print(X[:20])

[[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 ...

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False Fal

In [29]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=125, shuffle=True).history

Epoch 1/125
90/90 [==============================] - 13s 122ms/step - loss: 6.7681 - accuracy: 0.0465 - val_loss: 6.1332 - val_accuracy: 0.0745
Epoch 2/125
90/90 [==============================] - 11s 117ms/step - loss: 5.8915 - accuracy: 0.0877 - val_loss: 5.8890 - val_accuracy: 0.1275
Epoch 3/125
90/90 [==============================] - 10s 116ms/step - loss: 5.2446 - accuracy: 0.1359 - val_loss: 5.8214 - val_accuracy: 0.1192
Epoch 4/125
90/90 [==============================] - 11s 117ms/step - loss: 4.5460 - accuracy: 0.1991 - val_loss: 6.0676 - val_accuracy: 0.0877
Epoch 5/125
90/90 [==============================] - 10s 117ms/step - loss: 3.7887 - accuracy: 0.2924 - val_loss: 6.3224 - val_accuracy: 0.0728
Epoch 6/125
90/90 [==============================] - 10s 115ms/step - loss: 3.0559 - accuracy: 0.4218 - val_loss: 6.7380 - val_accuracy: 0.0596
Epoch 7/125
90/90 [==============================] - 10s 115ms/step - loss: 2.4703 - accuracy: 0.5324 - val_loss: 7.0757 - val_accuracy:

In [27]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

In [28]:
history

{'accuracy': [0.05411066487431526,
  0.09094082564115524,
  0.1350148320198059,
  0.1857217699289322,
  0.2672368586063385,
  0.371879905462265,
  0.48141035437583923,
  0.5826496481895447,
  0.6796125173568726,
  0.7545819282531738,
  0.8064234852790833,
  0.8470937609672546,
  0.8724035620689392,
  0.8924768567085266,
  0.9068772792816162,
  0.9183976054191589,
  0.9223250150680542,
  0.9297434091567993,
  0.9352417588233948,
  0.9350671768188477,
  0.9338453412055969,
  0.9328853487968445,
  0.933147132396698,
  0.9343690276145935,
  0.9314016699790955],
 'loss': [6.483112335205078,
  5.837347507476807,
  5.26887845993042,
  4.682735443115234,
  4.028784275054932,
  3.377232789993286,
  2.7777469158172607,
  2.2542479038238525,
  1.8111088275909424,
  1.4569590091705322,
  1.1947365999221802,
  0.9976547360420227,
  0.85457843542099,
  0.7630259394645691,
  0.6823788285255432,
  0.63102126121521,
  0.6010006666183472,
  0.5663062334060669,
  0.5420355796813965,
  0.5387985706329346,

In [36]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
        return x

In [37]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [38]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [48]:
q =  "one day i will beccome"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower()))
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  one day i will beccome
Sequence:  one day i will beccome
one
next possible words:  ['mother', 'again', 'him', 'woman', 'could']


In [51]:
q =  "nothing for the"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower()))
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  nothing for the
Sequence:  nothing for the
nothing
next possible words:  ['thing', 'sort', 'get', 'commented', 'tried']


In [52]:
q =  "Altogether"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower()))
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  Altogether
Sequence:  altogether
altogether
next possible words:  ['look', 'he', 'where', 'ash', 'okay']


In [53]:
q =  "hard to believe that"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower()))
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  hard to believe that
Sequence:  hard to believe that
hard
next possible words:  ['son', 'coughed', 'march', 'sharp', 'brush']


In [54]:
q =  "Now you must"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower()))
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  Now you must
Sequence:  now you must
now
next possible words:  ['of', 'covering', 'if', 'but', 'else']


In [56]:
q =  "Same day i tried to"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower()))
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  Same day i tried to
Sequence:  same day i tried to
same
next possible words:  ['bought', 'safe', 'standing', 'understood', 'god']
